In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.decomposition import PCA

## Simulate data

In [ ]:
def func_gauss(x, amp, mu, sigma):
    y = amp * np.sqrt(2*np.pi*sigma**2) ** -1 * \
        np.exp(-(x-mu)**2 / (2*sigma**2))
    return y

def func_gauss_linear(x, amp, mu, sigma, m, b):
    y = func_gauss(x, amp, mu, sigma) + m * x + b
    return y

def func_gauss_double_linear(x, amp1, mu1, sigma1, amp2, mu2, sigma2, m, b):
    y = (func_gauss_linear(x, amp1, mu1, sigma1, m, b) + \
         func_gauss(x, amp2, mu2, sigma2))
    return y

In [ ]:
def random_gauss_double_linear(x):
        amp = np.random.random(2)
        mu = np.random.random(2)*2
        sigma = np.random.random(2)
        m = (np.random.random(1)-0.5) / 10
        b = np.random.random(1)-0.5
        popt = [amp[0], mu[0], sigma[0], amp[1], -mu[1], sigma[1], m[0], b[0]]
        y = func_gauss_double_linear(x, *popt)
        
        return popt, y

In [ ]:
n = 4
fig, axs = plt.subplots(4,4,figsize=[10,10])
for i in range (n):
    for j in range (n):
        x = np.linspace(-3,3,1000)
        popt, y = random_gauss_double_linear(x)
        b = popt[7]
        #axs[i,j].set_title(list(np.round(popt, 3)))
        axs[i,j].scatter(x, y, alpha=1, s=1)
        axs[i,j].vlines(popt[1], b, popt[0]+b, color='C1')
        axs[i,j].vlines(popt[4], b, popt[3]+b, color='C2')
        axs[i,j].hlines(np.median(y), popt[1] - popt[2], popt[1] + popt[2], color='C1', alpha=0.5)
        axs[i,j].hlines(np.median(y), popt[4] - popt[5], popt[4] + popt[5], color='C2', alpha=0.5)
plt.tight_layout()

In [ ]:
popts = []
ys = []
for i in tqdm(range(100000)):
    popt, y = random_gauss_double_linear(x)
    popts.append(popt)
    ys.append(y)

In [ ]:
popts = np.array(popts)
ys = np.array(ys)

## PCA

In [ ]:
pca = PCA(n_components=2, whiten=True)

In [ ]:
pca_data = pca.fit_transform(ys)

In [ ]:
pca_data.shape

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
n = 2
titles = ['amp0', 'mu0', 'sigma0', 'amp1', 'mu1', 'sigma1', 'm', 'b']
fig, axs = plt.subplots(2,4,figsize=[20,10], sharex=True, sharey=True)
for i in range (2):
    for j in range (4):
        ind = i*4+j
        ax = axs[i, j]
        ax.set_title(titles[ind])
        pcm = ax.scatter(pca_data[:, 0], pca_data[:, 1], s=1, alpha=0.5, c=popts[:, ind])
        fig.colorbar(pcm, ax=ax)
plt.xlim(-n,n)
plt.ylim(-n,n)
plt.tight_layout()

In [ ]:
plt.plot(pca.components_[0])